In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# changed animal_shelter and AnimalShelter to match CRUD Python module file name and class name
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Ruby Bingham Dashboard'))),
    html.Hr(),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Div(dcc.RadioItems(id='filter-type',options=[
        {'label': 'Water Rescue', 'value': 'WR'},
        {'label': 'Mountain/Wilderness Rescue', 'value': 'MWR'},
        {'label': 'Disaster Rescue/Individual Tracking', 'value': 'DRIT'},
        {'label': 'Reset', 'value': 'RESET'}],
                            value='RESET',
                            labelStyle={'display': 'inline-block'})
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[],
                         page_action="native",
                         page_current=0,
                         page_size=10,),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    df = pd.DataFrame.from_records(db.read({}))
    if filter_type == 'WR':
        df = pd.DataFrame.from_records(db.read({'$and': [
            {'$or': [
                {'breed': 'Labrador Retriever Mix'},
                {'breed': 'Chesapeake Bay Retriever'},
                {'breed': 'Newfoundland'},
            ]},
            {'sex_upon_outcome': 'Intact Female'},
            {'age_upon_outcome_in_weeks': {'$gte': 26}},
            {'age_upon_outcome_in_weeks': {'$lte': 156}}
        ]}))
    elif filter_type == 'MWR':
        df = pd.DataFrame.from_records(db.read({'$and': [
            {'$or': [
                {'breed': 'German Shepherd'},
                {'breed': 'Alaskan Malamute'},
                {'breed': 'Old English Sheepdog'},
                {'breed': 'Siberian Husky'},
                {'breed': 'Rottweiler'}
            ]},
            {'sex_upon_outcome': 'Intact Male'},
            {'age_upon_outcome_in_weeks': {'$gte': 26}},
            {'age_upon_outcome_in_weeks': {'$lte': 156}}
        ]}))
    elif filter_type == 'DRIT':
        df = pd.DataFrame.from_records(db.read({'$and': [
            {'$or': [
                {'breed': 'Doberman Pinscher'},
                {'breed': 'German Shepherd'},
                {'breed': 'Golden Retriever'},
                {'breed': 'Bloodhound'},
                {'breed': 'Rottweiler'}
            ]},
            {'sex_upon_outcome': 'Intact Male'},
            {'age_upon_outcome_in_weeks': {'$gte': 20}},
            {'age_upon_outcome_in_weeks': {'$lte': 300}}
        ]}))
    elif filter_type == 'RESET':
        df = pd.DataFrame.from_records(db.read({}))
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')
    df.drop(columns=['_id'],inplace=True)
    return data

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if len(dff) == 0:
        return [dcc.Graph(figure={})]

    fig = px.pie(dff, names='breed', title='Animals by Breed')
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    if not dff.empty:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[30.75,-97.48], children=[
                    dl.Tooltip(dff.iloc[0, 4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[0, 9])
                    ])
                ])
            ])
        ]
    return None


app.run_server(debug=True)


Dash app running on http://127.0.0.1:17869/
